# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [27]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [28]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [29]:
# Import the necessary CSVs to Pandas DataFrames
sfo_data_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(sfo_data_path, index_col="year")

locations_path = Path("Data/neighborhoods_coordinates.csv")
neighborhood_locations = pd.read_csv(locations_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [30]:
housing_unit_by_year = sfo_data.groupby('year')
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    housing_unit = housing_unit_by_year['housing_units'].first()
    standard = housing_unit.std()
    return housing_unit.hvplot.bar(
        x="year",
        y="housing_units",
        xlabel='Year',
        ylabel='Housing Units',
        label=f"Housing Units in San Francisco from 2010 to 2016",
        ylim=(housing_unit.min()-standard,housing_unit.max()+standard)
    ).opts(yformatter="%.0f")


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    return housing_unit_by_year['gross_rent'].mean().hvplot(
        label='Average Gross Rent in San Francisco',
        xlabel='Year', ylabel='Gross Rent',
        xlim=(housing_unit_by_year.first().index.min(), housing_unit_by_year.first().index.max())
    )


def average_sales_price():
    """Average Sales Price Per Year."""
    
    return housing_unit_by_year['sale_price_sqr_foot'].mean().hvplot(
        label='Average Price per Sqft by Year in San Francisco',
        xlabel='Year', ylabel='Avg. Sale Price',
        xlim=(housing_unit_by_year.first().index.min(), housing_unit_by_year.first().index.max())
    )


neighborhood = sfo_data.groupby(['year','neighborhood']).mean().reset_index()

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    return neighborhood.hvplot(
        label='Average Price per Square feet by year',
        x='year',
        y='sale_price_sqr_foot',
        xlabel='Year',
        ylabel='Avg. Sales Price per Square Foot', 
        groupby='neighborhood'
    )

expensive_neighborhoods = sfo_data.groupby("neighborhood").mean().sort_values(ascending=False, by="sale_price_sqr_foot")

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    return expensive_neighborhoods['sale_price_sqr_foot'].head(10).hvplot.bar(
    ylabel='Average Sale Price per Square Foot',
    xlabel='Neighborhood',
    label='Top 10 Most Expensive Neighborhoods in San Francisco',
    rot=40,)

avg_neighborhood_value = neighborhood.drop(columns=['year']).groupby('neighborhood').mean().round(2)
most_expensive_neighborhood = avg_neighborhood_value.sort_values("sale_price_sqr_foot", ascending = False).head(10)
def parallel_categories():
    """Parallel Categories Plot."""
    
    return px.parallel_categories(
        most_expensive_neighborhood.reset_index(), 
        color='sale_price_sqr_foot',
        title='Parallel Categories Plot',
        color_continuous_scale=px.colors.cyclical.IceFire
    )

def most_expensive_neighborhoods_rent_sales():
    """Comparison of Rent and Sales Price of Most Expensive Neighborhoods"""
    return neighborhood[["sale_price_sqr_foot",'gross_rent','neighborhood','year']].hvplot.bar(
    xlabel = "Neighborhood",
    ylabel = "Num Housing Units",
    x= "year",
    label = "Top 10 Expensive Neighborhoods in SFO",
    groupby = "neighborhood",
    figsize = (20,20),
    rot = 60
)

def neighborhood_map():
    """Neighborhood Map"""
    avg_values_neighborhood = sfo_data.groupby(['neighborhood']).mean().reset_index().rename(columns = {'neighborhood' : 'Neighborhood'})
    new_avg_values_neighborhood=avg_values_neighborhood.reset_index().rename(columns = {'neighborhood':'Neighborhood'})
    neighborhood_location=pd.merge(neighborhood_locations,new_avg_values_neighborhood, on='Neighborhood')
    
    return px.scatter_mapbox(neighborhood_location,
                             lat="Lat",
                             lon="Lon",
                             size='sale_price_sqr_foot',
                             color='gross_rent',
                             hover_name='Neighborhood',
                             zoom=11,
                             color_continuous_scale=px.colors.cyclical.IceFire
                            )


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [31]:
location_column = pn.Column(' Real estate investment opportunities in San Fransco with interactive map with the average prices per neighborhood',
                           neighborhood_map())

yearly_market_analysis = pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())

neighborhood_analysis = pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods(), most_expensive_neighborhoods_rent_sales())

parallel_plot_analysis = pn.Column(parallel_categories())


panel_content = pn.Tabs(
    ('Welcome', location_column),
    ('Yearly Market Analysis', yearly_market_analysis),
    ('Neighborhood Analysis', neighborhood_analysis),
    ('Parallel Plot Analysis', parallel_plot_analysis)
)

panel = pn.Column(f'# Real Estate Analysis of San Francisco from {sfo_data.index.min()} to {sfo_data.index.max()}<br/>',
                  panel_content)
                 

## Serve the Panel Dashboard

In [32]:
# Serve the# dashboard
panel.servable()

Column
    [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Row
            [0] HoloViews(Bars)
            [1] HoloViews(Curve)
            [2] HoloViews(Curve)
        [2] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] HoloViews(Bars)
            [2] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
        [3] Column
            [0] Plotly(Figure)